In [7]:
# !pip install opencv-python
!pip install tensorflow

  Using cached tensorflow-2.12.0-cp311-cp311-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp311-cp311-win_amd64.whl (272.9 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.9.0-cp311-cp311-win_amd64.whl (2.7 MB)
  Using cached jax-0.4.13-py3-none-any.whl
  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl (14.6 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.23.3-cp310-abi3-win_amd64.whl (422 kB)
  Using cached grpcio-1.56.0-cp311-cp311-win_amd64.whl (4.2 MB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp311-cp311-win_amd64.whl (1.5 MB

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import cv2
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:


# Load the model
model = tf.saved_model.load('/path/to/yolo')

# Preprocess the image
input_image = preprocess(frame)

# Run the model
outputs = model(input_image)

# Postprocess the outputs
boxes, scores, classes = postprocess(outputs)


In [1]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    cv2.imshow('Our Webcam Video', frame)
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()
